In [4]:
!pip install numba==0.48

In [5]:
!pip install regain

In [6]:
%matplotlib inline
import time
import numpy as np
import pandas as pd

# Sample data

In [7]:
X = np.array([[0.8, 0.0, 0.2, 0.0],
                     [0.0, 0.4, 0.0, 0.0],
                     [0.2, 0.0, 0.3, 0.1],
                     [0.0, 0.0, 0.1, 0.7]])

In [8]:
# np.random.seed(243)
# X = np.random.multivariate_normal(mean=[0, 0, 0, 0], cov=true_cov, size=200)
# X

# Best model solution Z

Assume found parameters with GridSearch

In [109]:
import sys
sys.path.append('..')
# sys.path.append(sys.path[0] + "/..")

from gglasso.solver.single_admm_solver import ADMM_SGL

In [110]:
sol, info = ADMM_SGL(X, lambda1, Omega_0 , eps_admm = 1e-10, verbose = False, latent = False)

ADMM terminated after 388 iterations with accuracy 9.881716810385407e-11
ADMM status: optimal


In [112]:
Z = np.array(sol["Theta"])

# Import sklearn

In [113]:
from sklearn.covariance import GraphicalLasso
from sklearn.covariance import empirical_covariance

In [114]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    rg = GraphicalLasso(tol=tol, max_iter = 1000).fit(X)
    end = time.time()
    cov_list.append(rg.covariance_)
    precision_list.append(rg.precision_)
    time_list.append(end-start)

In [115]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [117]:
method_list = ["sklearn"]*len(accuracy_list)

In [118]:
df_sk = pd.DataFrame(data={'time': time_list, 
                           'accuracy': accuracy_list,
                           'method':method_list})
df_sk.head()

,time,accuracy,method
0,0.005764,-96.418322,sklearn
1,0.001652,-96.418322,sklearn
2,0.002085,-96.418350,sklearn
3,0.002534,-96.418354,sklearn


# Import Regain GL

In [119]:
from regain.covariance import GraphicalLasso
#from regain.covariance.graphical_lasso_ import GraphicalLasso

In [120]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    rg = GraphicalLasso(alpha=0.01, rho=1.,tol=tol, rtol=1e-7, max_iter=1000, verbose=False).fit(X)
    end = time.time()
    cov_list.append(rg.covariance_)
    precision_list.append(rg.precision_)
    time_list.append(end-start)

In [121]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [122]:
method_list = ["regain"]*len(accuracy_list)

In [123]:
df_rg = pd.DataFrame(data={'time': time_list, 
                           'accuracy': accuracy_list,
                           'method':method_list})
df_rg.head()

,time,accuracy,method
0,0.074993,-99.118805,regain
1,0.058203,-96.704424,regain
2,0.071683,-96.447073,regain
3,0.076359,-96.421230,regain


# Import ADMM SGL

In [124]:
#from helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix

In [125]:
import sys
sys.path.append('..')
# sys.path.append(sys.path[0] + "/..")

from gglasso.solver.single_admm_solver import ADMM_SGL

In [126]:
Omega_0 = np.eye(X.shape[0])
lambda1 = 0.01

In [127]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    sol, info = ADMM_SGL(X, lambda1, Omega_0 , eps_admm = tol, verbose = False, latent = False)
    end = time.time()
    cov_list.append(sol['Omega'])
    precision_list.append(sol['Theta'])
    time_list.append(end-start)

ADMM terminated after 101 iterations with accuracy 9.667825867681589e-05
ADMM status: optimal
ADMM terminated after 149 iterations with accuracy 9.575201951721199e-06
ADMM status: optimal
ADMM terminated after 197 iterations with accuracy 9.538835394609324e-07
ADMM status: optimal
ADMM terminated after 244 iterations with accuracy 9.975989612201002e-08
ADMM status: optimal


In [128]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [129]:
method_list = ["admm"]*len(accuracy_list)

In [130]:
df_admm = pd.DataFrame(data={'time': time_list, 
                             'accuracy': accuracy_list,
                             'method':method_list})
df_admm.head()

,time,accuracy,method
0,0.019372,0.986726,admm
1,0.022584,0.998680,admm
2,0.031264,0.999868,admm
3,0.034597,0.999986,admm


# Plotting

In [ ]:
# !pip install plotly

In [149]:
frames = [df_sk, df_rg, df_admm]

df = pd.concat(frames)
df

,time,accuracy,method
0,0.005764,-96.418322,sklearn
1,0.001652,-96.418322,sklearn
2,0.002085,-96.418350,sklearn
3,0.002534,-96.418354,sklearn
0,0.074993,-99.118805,regain
1,0.058203,-96.704424,regain
2,0.071683,-96.447073,regain
3,0.076359,-96.421230,regain
0,0.019372,0.986726,admm
1,0.022584,0.998680,admm


In [150]:
#normalised accuracy [0,1]
norm_ac = np.array(df['accuracy'])
df['accuracy'] = (a - np.min(norm_ac))/np.ptp(norm_ac)
df

,time,accuracy,method
0,0.005764,0.990070,sklearn
1,0.001652,0.990029,sklearn
2,0.002085,0.990033,sklearn
3,0.002534,0.990037,sklearn
0,0.074993,0.990761,regain
1,0.058203,0.990593,regain
2,0.071683,0.990728,regain
3,0.076359,0.990775,regain
0,0.019372,0.990205,admm
1,0.022584,0.990238,admm


In [151]:
import plotly.express as px

fig = px.line(df, x="time", y="accuracy", color= "method", title='Perfomance comparison')
fig.show()

# Import LatentTimeGraphicalLasso

In [ ]:
from regain.covariance import LatentTimeGraphicalLasso

In [ ]:
from regain.covariance import latent_graphical_lasso_, latent_time_graphical_lasso_

In [ ]:
np.random.seed(243)

tau = 0.1 #mu1 in ADMM_SGL
alpha = 0.1 # lambda in ADMM_SGL

In [ ]:
sol = latent_graphical_lasso_.latent_graphical_lasso(
    true_cov, alpha=alpha, tau=tau, tol=1e-5, rtol=1e-5,
    rho=1. / true_cov.shape[0], verbose=0, max_iter=500)

In [ ]:
prec, cov, n_iter = sol[0], sol[1], sol[2]

In [ ]:
prec,

In [ ]:
cov